In [95]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.decomposition import PCA

In [79]:
df = pd.read_csv('car_prices.csv')
df.shape

(558837, 16)

In [80]:
df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [81]:
df.drop(['vin', 'saledate'], axis='columns', inplace=True)
df.isna().sum()

year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
state               0
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
dtype: int64

In [82]:
df['condition'].fillna(df['condition'].median(), inplace=True)
df['odometer'].fillna(df['odometer'].median(), inplace=True)
df.isna().sum()

year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
state               0
condition           0
odometer            0
color             749
interior          749
seller              0
mmr                38
sellingprice       12
dtype: int64

In [83]:
df.dropna(inplace=True)
df.shape

(481747, 14)

In [84]:
df.describe()

,year,condition,odometer,mmr,sellingprice
count,481747.000000,481747.000000,481747.000000,481747.000000,481747.000000
mean,2010.122517,30.856383,67695.768648,13702.387197,13550.306076
std,3.879080,13.169549,52508.800656,9535.884430,9613.683211
min,1990.000000,1.000000,1.000000,25.000000,1.000000
25%,2008.000000,24.000000,28441.000000,7200.000000,7000.000000
50%,2012.000000,35.000000,52078.000000,12200.000000,12100.000000
75%,2013.000000,41.000000,98196.000000,18150.000000,18000.000000
max,2015.000000,49.000000,999999.000000,182000.000000,230000.000000


In [85]:
year_upper = df['year'].mean() + 3 * df['year'].std()
year_lower = df['year'].mean() - 3 * df['year'].std()
condition_upper = df['condition'].mean() + 3 * df['condition'].std()
condition_lower = df['condition'].mean() - 3 * df['condition'].std()
odometer_upper = df['odometer'].mean() + 3 * df['odometer'].std()
odometer_lower = df['odometer'].mean() - 3 * df['odometer'].std()
mmr_upper = df['mmr'].mean() + 3 * df['mmr'].std()
mmr_lower = df['mmr'].mean() - 3 * df['mmr'].std()
sellingprice_upper = df['sellingprice'].mean() + 3 * df['sellingprice'].std()
sellingprice_lower = df['sellingprice'].mean() - 3 * df['sellingprice'].std()

year_outliers = df[(df['year'] < year_lower) | (df['year'] > year_upper)]
condition_outliers = df[(df['condition'] < condition_lower) | (df['condition'] > condition_upper)]
odometer_outliers = df[(df['odometer'] < odometer_lower) | (df['odometer'] > odometer_upper)]
mmr_outliers = df[(df['mmr'] < mmr_lower) | (df['mmr'] > mmr_upper)]
sellingprice_outliers = df[(df['sellingprice'] < sellingprice_lower) | (df['sellingprice'] > sellingprice_upper)]

outliers = pd.concat([year_outliers, condition_outliers, odometer_outliers, mmr_outliers, sellingprice_outliers])
outliers.drop_duplicates(inplace=True)

df.drop(outliers.index, axis='index', inplace=True)

outliers.shape, df.shape

((14916, 14), (466831, 14))

In [87]:
le = LabelEncoder()
df['make'] = le.fit_transform(df['make'])
df['model'] = le.fit_transform(df['model'])
df['trim'] = le.fit_transform(df['trim'])
df['body'] = le.fit_transform(df['body'])
df['transmission'] = le.fit_transform(df['transmission'])
df['state'] = le.fit_transform(df['state'])
df['color'] = le.fit_transform(df['color'])
df['interior'] = le.fit_transform(df['interior'])
df['seller'] = le.fit_transform(df['seller'])
df.head()

,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,sellingprice
0,2015,21,563,745,33,0,2,5.0,16639.0,17,1,6406,20500.0,21500.0
1,2015,21,563,745,33,0,2,5.0,9393.0,17,0,6406,20800.0,21500.0
2,2014,3,6,234,34,0,2,45.0,1331.0,7,1,4418,31900.0,30000.0
3,2015,45,508,1112,34,0,2,41.0,14282.0,17,1,12296,27500.0,27750.0
5,2015,32,47,95,34,0,2,1.0,5554.0,7,1,4091,15350.0,10900.0


In [88]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
df_scaled[0]

array([1.        , 0.45652174, 0.82309942, 0.54339898, 0.40740741,
       0.        , 0.06060606, 0.08333333, 0.07387411, 0.89473684,
       0.0625    , 0.50405225, 0.4843288 , 0.50826261])

In [89]:
df_scaled[0:2]

array([[1.        , 0.45652174, 0.82309942, 0.54339898, 0.40740741,
        0.        , 0.06060606, 0.08333333, 0.07387411, 0.89473684,
        0.0625    , 0.50405225, 0.4843288 , 0.50826261],
       [1.        , 0.45652174, 0.82309942, 0.54339898, 0.40740741,
        0.        , 0.06060606, 0.08333333, 0.04170126, 0.89473684,
        0.        , 0.50405225, 0.49142519, 0.50826261]])

In [90]:
X = df_scaled[:, :-1]
y = df_scaled[:, -1]
X[0], y[0]

(array([1.        , 0.45652174, 0.82309942, 0.54339898, 0.40740741,
        0.        , 0.06060606, 0.08333333, 0.07387411, 0.89473684,
        0.0625    , 0.50405225, 0.4843288 ]),
 0.508262606681009)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [97]:
model_params = {
    'Linear': {
        'model': LinearRegression(),
        'params': {}
    },
    'Lasso': {
        'model': Lasso(),
        'params': {
            'alpha': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
        }
    },
    'Ridge': {
        'model': Ridge(),
        'params': {
            'alpha': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1],
            'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
        }
    },
    'Ridge lbfgs': {
        'model': Ridge(positive=True, solver='lbfgs'),
        'params': {
            'alpha': [1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
        }
    }
}

scores = []

for mn, mp in model_params.items():
    gscv = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    gscv.fit(X, y)

    scores.append({
        'Model': mn,
        'Best Params': gscv.best_params_,
        'Best Score': gscv.best_score_
    })

no_pca_scores = pd.DataFrame(scores, columns=['Model', 'Best Params', 'Best Score'])
no_pca_scores

,Model,Best Params,Best Score
0,Linear,{},0.962629
1,Lasso,{'alpha': 1e-05},0.962645
2,Ridge,"{'alpha': 0.001, 'solver': 'sag'}",0.962630
3,Ridge lbfgs,{'alpha': 1e-10},0.962539


In [99]:
pca = PCA(0.95)
pca_X = pca.fit_transform(X)

scores = []

for mn, mp in model_params.items():
    gscv = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    gscv.fit(pca_X, y)

    scores.append({
        'Model': mn,
        'Best Params': gscv.best_params_,
        'Best Score': gscv.best_score_
    })

pca_scores = pd.DataFrame(scores, columns=['Model', 'Best Params', 'Best Score'])
pca_scores

,Model,Best Params,Best Score
0,Linear,{},0.581557
1,Lasso,{'alpha': 0.001},0.632119
2,Ridge,"{'alpha': 1, 'solver': 'saga'}",0.581607
3,Ridge lbfgs,{'alpha': 1},-0.043668
